In [60]:
import numpy as np

# Chain Drive Design Script

Follow the instructions. Thank Felix.

Felix takes no responsability for marks lost due to errors in this notebook, Fuck you.

In [61]:
# Fill in these constants
P_in = 10   # horsepower
n1 = 1500   # RMP
n2 = 500    # RPM

# if you know VR but not n1 or n2, adjust accordingly
VR = n1 / n2

print(f'Velocity Ratio: {VR}')

Velocity Ratio: 3.0


In [62]:
# use this function to linearly interpolate from graphs
# known points are (x1, y1) and (x2, y2)
# function returns the interpolated value associated with x3
def linear_interpolate(x1, y1, x2, y2, x3):
    # Ensure x1 and x2 are not the same to avoid division by zero
    if x1 == x2:
        raise ValueError("x1 and x2 cannot be the same value.")
    
    y3 = y1 + (y2 - y1) * ((x3 - x1) / (x2 - x1))
    return y3

### Behold the chains

If these chains are used to support a load or apply a tensile force, only 10% of the average tensile
strength should be used.

![V Belt Image](./images/chain_drive_types.png)


### Get a service factor

![V Belt Image](./images/chain_service_factor.png)

In [63]:
# FILL IN ______________________________
SF = 1.3

P_des = P_in * SF

print(f'Design Power: {P_des} HP')

Design Power: 13.0 HP


### Time to Select a Chain

We are selecting for the chain pitch (p) and the number of teeth on the small sprocket (N1).

In [64]:
# set true the one you wanna use
# SET THE OTHERS FALSE OR SUFFER
one_stranded = True
two_stranded = False
three_stranded = False
four_stranded = False

strand_factor = 1

if one_stranded:
    print('you are using one strand')
    strand_factor = 1
elif two_stranded:
    print('you are using two strands')
    strand_factor = 1.7
elif three_stranded:
    print('you are using three strands')
    strand_factor = 2.5
elif four_stranded:
    print('you are using four strands')
    strand_factor = 3.3

P_required = P_des / strand_factor

print(f'Required power is: {P_required} HP')

you are using one strand
Required power is: 13.0 HP


### Keep the following in mind: 

1. P_rated obtained from the tables must be greater than P_required
2. The minimum number of teeth in a sprocket should be 17 (unless the it is operation at < 100rpm)
3. The largest sprocket should have no more than 120 teeth, so make sure that (N1)(VR) < 120 when selecting N1.
4. You will sadly need to use interpolation to find the rated power if n1 isn’t on the table already.
5. The table will also give you the lubrication type

![Chain Image](./images/chain_No40.png)
![Chain Image](./images/chain_No60.png)
![Chain Image](./images/chain_No80.png)

In [65]:
# FILL IN ______________________________
N1 = 17
P_rated = 40
pitch = 1
lubrication_type = "Type C: Oil"

In [66]:
N2 = np.round(N1 * VR)

print(f'N2 = {N2} teeth')

n2 = n1 * N1 / N2

print(f'actual output speed n2 = {n2} RPM')

N2 = 51.0 teeth
actual output speed n2 = 500.0 RPM


### Now we compute the pitch diameters of the sprockets
![Chain Image](./images/chain_pitch_diameters.png)

In [67]:
rad1 = np.radians(180 / N1)
rad2 = np.radians(180 / N2)

PD1 = pitch / np.sin(rad1)

PD2 = pitch / np.sin(rad2)

print(f'PD1 = {PD1} inches')
print(f'PD2 = {PD2} inches')

PD1 = 5.442191151751806 inches
PD2 = 16.244075384859062 inches


### Specify the nominal CD. 
The recommended range is 30 to 50 pitches, so let’s specify 40 pitches.

In [68]:
# adjust as necessary
CD_nom = 40

### Now we find the required chain length in pitches
![Chain Image](./images/chain_length.png)

The chain length must be an integer multiple of the pitch, so round to the nearest integer
value

In [69]:
Lc = 2*CD_nom + (N1 + N2) / 2 + (N2 - N1)**2 / (np.pi**2 * 4 * CD_nom)

Lc = np.round(Lc)

print(f'Lc = {Lc} pitches')

Lc = 115.0 pitches


### Now we find the actual CD in pitches
![Chain Image](./images/chain_CD.png)

In [73]:
CD = (1 / 4)*(Lc - (N2 + N1) / 2 + np.sqrt((Lc - (N2 + N1)/2)**2 - 8*(N2 - N1)**2 / (4 * np.pi**2)))

print(f'actual CD = {CD} pitches')

actual CD = 40.13521030725303 pitches


### Now we calculate the wrap angle

![Chain Image](./images/chain_wrap_angle.png)

The minimum angle of wrap should be 120◦

In [71]:
phi_1 = 180 - 2*np.degrees(np.arcsin((PD2 - PD1)/(2 * CD)))

phi_2 = 180 + 2*np.degrees(np.arcsin((PD2 - PD1)/(2 * CD)))

if phi_1 < 120:
    print(f'OH SHIT, wrap angle of small sheave is {phi_1} which is less then 120! You fucked up!')

print(f'phi_1 = {phi_1} degrees')

print(f'phi_2 = {phi_2} degrees')

phi_1 = 164.53264146838927 degrees
phi_2 = 195.46735853161073 degrees


### Finally, factor of safety calculations!

In [74]:
P_allowed = P_rated * strand_factor

FS = P_allowed / P_des

print(f'Factor of safety is {FS}')

Factor of safety is 3.076923076923077
